використаємо очищений датасет з минулої ЛР

In [24]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/DA_kpi/text_cleand.csv')

In [25]:
df

,Unnamed: 0,title,text,label
0,0,smell hillarys fear,daniel greenfield shillman journalism fellow f...,0
1,1,watch exact moment paul ryan committed politic...,google pinterest digg linkedin reddit stumbleu...,0
2,2,kerry go paris gesture sympathy,us secretary state john f kerry said monday st...,1
3,3,bernie supporters twitter erupt anger dnc trie...,kaydee king kaydeeking november 9 2016 lesson...,0
4,4,battle new york primary matters,primary day new york frontrunners hillary clin...,1
...,...,...,...,...
6330,6330,state department says cant find emails clinton...,state department told republican national comm...,1
6331,6331,p pbs stand plutocratic pentagon,p pbs stand plutocratic pentagon posted oct 27...,0
6332,6332,antitrump protesters tools oligarchy info...,antitrump protesters tools oligarchy reform a...,0
6333,6333,ethiopia obama seeks progress peace security e...,addis ababa ethiopia president obama convened ...,1


In [26]:
fake_news = df[df['label'] == 0]

In [27]:
print(fake_news.isna().sum())

Unnamed: 0    0
title         1
text          0
label         0
dtype: int64


In [28]:
fake_news.dropna(inplace=True)

<ipython-input-28-f65758bb61c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_news.dropna(inplace=True)


In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text
tokenizer = Tokenizer(char_level=False)
tokenizer.fit_on_texts(fake_news['title'])
total_words = len(tokenizer.word_index) + 1

In [30]:
# Create input sequences and labels
input_sequences = []
for line in fake_news['title']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [31]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
677/677 [==============================] - 23s 31ms/step - loss: 8.5672 - accuracy: 0.0137
Epoch 2/100
677/677 [==============================] - 21s 31ms/step - loss: 8.0668 - accuracy: 0.0157
Epoch 3/100
677/677 [==============================] - 22s 32ms/step - loss: 7.8205 - accuracy: 0.0229
Epoch 4/100
677/677 [==============================] - 21s 31ms/step - loss: 7.4715 - accuracy: 0.0293
Epoch 5/100
677/677 [==============================] - 21s 30ms/step - loss: 7.0165 - accuracy: 0.0408
Epoch 6/100
677/677 [==============================] - 21s 31ms/step - loss: 6.5029 - accuracy: 0.0593
Epoch 7/100
677/677 [==============================] - 21s 32ms/step - loss: 5.9773 - accuracy: 0.0849
Epoch 8/100
677/677 [==============================] - 20s 30ms/step - loss: 5.4654 - accuracy: 0.1256
Epoch 9/100
677/677 [==============================] - 21s 32ms/step - loss: 4.9711 - accuracy: 0.1845
Epoch 10/100
677/677 [==============================] - 22s 33ms/step - l

In [36]:
# Function to generate text given a seed text
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        predicted_index = np.argmax(predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generate text
generated_text = generate_text("Trump", 5, model, max_sequence_length)
print("Generated Text:", generated_text)

Generated Text: Trump supporter arrested voting twiceto fight
